In [39]:
import os
import json

In [40]:
def flatten(l):
    fl = [item for sublist in l for item in sublist]
    return fl

In [ ]:
# import numpy
# l = numpy.array_split(remaining_titles,len(remaining_titles)/20);

In [41]:
datasets_path = "/Users/sophierand/RCDatasets/datasets.json"
with open(datasets_path) as json_file:
    datasets = json.load(json_file)

In [42]:
sed_datasets = [{'id':d['id'],'search_terms':[d['title']] + d['alt_title']} for d in datasets if d['id'] == 'dataset-370']
sed_datasets

[{'id': 'dataset-370',
  'search_terms': ['Survey of Earned Doctorates', 'SED', 'SESTAT']}]

In [43]:
import dimensions_search_api_client as dscli
import RichContextAPI

In [44]:
api_client = RichContextAPI.connect_dimensions_api()

In [45]:
def run_exact_string_search(string,api_client):
    search_string = 'search publications in full_data for "\\"{}\\"" return publications[doi+title+journal+author_affiliations]'.format(string)
    api_response = api_client.execute_query(query_string_IN = search_string )
    return api_response

In [52]:
md_list = []
ds_id = sed_datasets[0]['id']
search_terms = sed_datasets[0]['search_terms']
author_values = []
for i in search_terms:
    dimension_return = run_exact_string_search(string = i,api_client = api_client)
    dimension_return_pub = dimension_return['publications']
    ds_pub_list = []
    for d in dimension_return_pub:
        try:
            md_dict = {'title':d['title'],'journal':d['journal']['title'],'search_string':i,'doi':d['doi'],'datasets':ds_id}
            if 'author_affiliations' in d.keys():
                author_list = d['author_affiliations'][0]
                for a in author_list:
                    auth_dict = {'first_name':a['first_name'],'last_name':a['last_name'],'affiliation':a['affiliations'][0]['name']}
                    author_values.append(auth_dict)
                md_dict.update({'authors':author_values})
            elif 'author_affiliations' not in d.keys():
                pass
            ds_pub_list.append(md_dict)
        except:
            pass
    md_list.append(ds_pub_list)

In [53]:
md_list = flatten(md_list)

In [54]:
len(md_list)

57

In [55]:
md_list

[{'title': 'Does employment during doctoral training reduce the PhD completion rate?',
  'journal': 'Studies in Higher Education',
  'search_string': 'Survey of Earned Doctorates',
  'doi': '10.1080/03075079.2019.1672648',
  'datasets': 'dataset-370',
  'authors': [{'first_name': 'Saule',
    'last_name': 'Bekova',
    'affiliation': 'National Research University Higher School of Economics'},
   {'first_name': 'Todd R.',
    'last_name': 'Jones',
    'affiliation': 'Cornell University'},
   {'first_name': 'Ronald G.',
    'last_name': 'Ehrenberg',
    'affiliation': 'Cornell University'},
   {'first_name': 'Sweeney',
    'last_name': 'Windchief',
    'affiliation': 'Montana State University'},
   {'first_name': 'Donald G.',
    'last_name': 'Hackmann',
    'affiliation': 'University of Illinois at Urbana Champaign'},
   {'first_name': 'Joel R.',
    'last_name': 'Malin',
    'affiliation': 'Miami University'},
   {'first_name': 'Blakely',
    'last_name': 'Brown',
    'affiliation': 'U

In [50]:
file_name = 'SED_dimensions_linkages.json'

In [51]:
with open(file_name, 'w') as outfile:
    json.dump(md_list, outfile, indent = 2)

In [58]:
keys = ['title','doi','journal','search_string','datasets']

md_list_csv = [{k:m[k] for k in keys} for m in md_list]

In [61]:
import pd
md_df_csv = pd.DataFrame.from_dict(md_list_csv)

ModuleNotFoundError: No module named 'pd'